In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import sklearn as sk
from sklearn import decomposition as dec
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.utils.prune as prune
import sklearn.manifold as nonlin
import copy
import pickle as pk


In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

batch_size = 4
#cifar10
trainset_cifar = torchvision.datasets.CIFAR10(root='./data', train=True,
                                              download=True, transform=transform)
trainset_cifar, validset_cifar = torch.utils.data.random_split(trainset_cifar, [
                                                               45000, 5000])


trainloader_cifar = torch.utils.data.DataLoader(trainset_cifar, batch_size=batch_size,
                                                shuffle=True, num_workers=2)
validloader_cifar = torch.utils.data.DataLoader(validset_cifar, batch_size=batch_size,
                                                shuffle=True, num_workers=2)

testset_cifar = torchvision.datasets.CIFAR10(root='./data', train=False,
                                             download=True, transform=transform)
testloader_cifar = torch.utils.data.DataLoader(testset_cifar, batch_size=batch_size,
                                               shuffle=False, num_workers=2)

classes_cifar = ('plane', 'car', 'bird', 'cat',
                 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


# MNIST DATASET
mnist_train = torchvision.datasets.MNIST(root='./data', train=True,
                                         download=True, transform=transform)
mnist_train, mnist_valid = torch.utils.data.random_split(mnist_train, [
                                                         50000, 10000])
mnist_test = torchvision.datasets.MNIST(root='./data', train=False,
                                        download=True, transform=transform)

trainloader_mnist = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size,
                                                shuffle=True, num_workers=2)
validloader_mnist = torch.utils.data.DataLoader(mnist_valid, batch_size=batch_size,
                                                shuffle=True, num_workers=2)
testloader_mnist = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size,
                                               shuffle=True, num_workers=2)
classes_mnist = ('0', '1', '2', '3',
                 '4', '5', '6', '7', '8', '9')


Files already downloaded and verified
Files already downloaded and verified


In [4]:
mnist_dataset_sample = iter(testloader_mnist)
sample_im, sample_lbl = mnist_dataset_sample.next()

cifar_dataset_sample = iter(testloader_cifar)
sample_im_c, sample_lbl_c = cifar_dataset_sample.next()


# function to use later 

In [5]:
def are_weights(model,weights):
    w1,w2,w3 = weights
    model.fc1.weight.data = w1
    model.fc2.weight.data = w2
    model.fc3.weight.data = w3
    return model


def construct_model(model, layers):
    l1, l2, l3 = layers
    model.fc1 = l1
    model.fc2 = l2
    model.fc3 = l3
    return model
    
def change_dimensionality(weight, dr_method):
    x1 = weight.detach().numpy().T
    x1 = dr_method.fit_transform(x1)
    x1 = torch.tensor(x1.T, dtype=torch.float32)
    print("x_new vector")
    print(x1.shape)
    return x1

    

In [6]:
def drprojection(model, method, whiten):
    weight_list = []
    weight1 = model.fc1.weight
    print('fc1 layer weight')

    fc1_reduced = change_dimensionality(
        weight1, method(n_components=int(weight1.shape[0]/2),  whiten=whiten, random_state=0, tol= 10))
    weight_list.append(fc1_reduced)
    x2 = torch.mm(model.fc1.weight, fc1_reduced.T)
    print(x2.shape)
    weight2 = torch.mm(model.fc2.weight, x2)
    print(weight2.shape)

    fc2_reduced = change_dimensionality(
        weight2, method(n_components=int(weight2.shape[0]/2),whiten=whiten, random_state=0, tol= 10))
    print('fc2 layer weight')
    print(fc2_reduced.shape)
    weight_list.append(fc2_reduced)

    x3_reduced = torch.mm(weight2, fc2_reduced.T)

    fc3_reduced = torch.mm(model.fc3.weight, x3_reduced)
    print('fc3 layer weight')
    print(fc3_reduced.shape)
    weight_list.append(fc3_reduced)

    return weight_list


In [7]:
def average_accuracy(net, dataset):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in dataset:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(
        f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')


In [8]:
def every_class_accuracy(model, testloader, classes):
    cor_pred = {classname: 0 for classname in classes}
    t_pred = {classname: 0 for classname in classes}
    with torch.no_grad():
        for data in testloader:
            im, labels = data
            output = model(im)
            _, predictions = torch.max(output, dim=1)
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    cor_pred[classes[label]] += 1
                t_pred[classes[label]] += 1
    
    for classname, correct_count in cor_pred.items():
        accuracy = 100 * float(correct_count) / t_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

        

In [23]:
from torch.profiler import profile, record_function, ProfilerActivity
import torch.optim as optim


def training(model, trainset,valset, n, filename, is_dr):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    running_loss = 0.0
    total = 0.0
    correct = 0.0
    loss_nodr = []
    acc_nodr = []
    
    val_loss_nodr = []

    val_acc_nodr = []
  

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    print(optimizer)

    for epoch in range(n):

        print('epoch:', epoch)
        for i, data in enumerate(trainset):

            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad(True)
            with torch.set_grad_enabled(True):
                with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
                    if is_dr == True:
                        outputs = nn.functional.normalize(model(inputs).to(device))
                    else:
                        outputs = model(inputs).to(device)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                running_loss += loss.item()
        model.eval()
        val_loss = 0.0
        total_val=0.0
        correct_val = 0.0

        for i, data in enumerate(valset):

            inputs, labels2 = data
            inputs = inputs.to(device)
            labels2 = labels2.to(device)
            if is_dr == True:
                        val_output = nn.functional.normalize(model(inputs).to(device))
            else:
                        val_output = model(inputs).to(device)
            loss = criterion(val_output, labels2)
            val_loss+=loss.item()
            total_val+=labels2.size(0)
            _, pred = torch.max(val_output.data, 1)
            correct_val += (pred == labels2).sum().item()

            
            
        train_loss = running_loss/len(trainset)
        acc_temp = 100 * correct / total

        valid_loss_temp = val_loss/len(valset)
        valid_acc_temp = (100 * correct_val)/total_val
        loss_nodr.append(train_loss)
        acc_nodr.append(acc_temp)

        val_loss_nodr.append(valid_loss_temp)
        val_acc_nodr.append(valid_acc_temp)
        print(
            f'[{epoch + 1}, {i + 1:5d}] train loss: {train_loss:.3f} train acc: {acc_temp:.3f}', 
            f'valid acc: {valid_acc_temp:.3f} valid loss  {valid_loss_temp:.3f} ')
        
        running_loss = 0.0
        correct = 0.0
        total = 0.0

    print('Finished Training')
    pickle_accloss(acc_nodr, loss_nodr,val_acc_nodr,val_loss_nodr,  filename)

    
def pickle_accloss(acc, loss,valid_acc, valid_loss, filename):
    accandloss = {'accuracy' : acc, 'loss' : loss, 'valid acc' : valid_acc, 'valid loss': valid_loss}
    with open('./data_vis/PCA/{}.pickle'.format(filename), 'wb') as ica_acc_and_loss:
        pk.dump(accandloss, ica_acc_and_loss)
    print('Saved dictionary of loss and accuracy!')



# Dimensional reduction on Mnist dataset

In [11]:
class NN(nn.Module):
    def __init__(self, is_normalize):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=1, out_channels=12, kernel_size=(5, 5))
        self.pool = nn.MaxPool2d(kernel_size=(2), stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=12, out_channels=16, kernel_size=(5, 5))
        self.fc1 = nn.Linear(256, 120) # 256x 60
        self.fc2 = nn.Linear(120, 84) # 60 x 84
        self.fc3 = nn.Linear(84, 10)
        self.soft = nn.Softmax(dim=0)
        self.norm = is_normalize

        # self.conv1 = nn.Conv2d(3, 6, 5)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = nn.Conv2d(6, 16, 5)
        # self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # self.fc2 = nn.Linear(120, 84)
        # self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        if(self.norm==True):
         x = nn.functional.normalize(self.fc3(x))
        else:
         x = self.fc3(x)
         
        # x = self.soft(x)
        return x
    

### Training of the network

TESTING NETWORK WITH DIMEMSNIOALLY REDUCED NETWORK 

In [12]:
path_original_network_mnist = './pca_reduced_networks/mnist/original_network_mnist.pt'
original_network_mnist = torch.load(path_original_network_mnist)


In [13]:
average_accuracy(original_network_mnist, testloader_mnist)


Accuracy of the network on the 10000 test images: 99 %


In [13]:
every_class_accuracy(original_network_mnist, testloader_mnist, classes_mnist)

Accuracy for class: 0     is 99.2 %
Accuracy for class: 1     is 99.5 %
Accuracy for class: 2     is 99.0 %
Accuracy for class: 3     is 98.6 %
Accuracy for class: 4     is 98.8 %
Accuracy for class: 5     is 99.2 %
Accuracy for class: 6     is 98.7 %
Accuracy for class: 7     is 99.5 %
Accuracy for class: 8     is 99.5 %
Accuracy for class: 9     is 98.1 %


## dimensional reduction of the original network without retraining

In [30]:
pca_mnist_network = copy.deepcopy(original_network_mnist)

mnist_weight_list = drprojection(
    pca_mnist_network, dec.PCA, 'unit-variance')

pca_mnist_layers = (nn.Linear(256, 60, False), nn.Linear(
    60, 42, False), nn.Linear(42, 10, False))

pca_mnist_network = construct_model(
    pca_mnist_network, pca_mnist_layers)
    
pca_mnist_network = are_weights(pca_mnist_network, tuple(mnist_weight_list))
pca_mnist_network.eval()


fc1 layer weight
x_new vector
torch.Size([60, 256])
torch.Size([120, 60])
torch.Size([84, 60])
x_new vector
torch.Size([42, 60])
fc2 layer weight
torch.Size([42, 60])
fc3 layer weight
torch.Size([10, 42])


NN(
  (conv1): Conv2d(1, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=60, bias=False)
  (fc2): Linear(in_features=60, out_features=42, bias=False)
  (fc3): Linear(in_features=42, out_features=10, bias=False)
  (soft): Softmax(dim=0)
)

In [31]:
average_accuracy(pca_mnist_network, testloader_mnist)


Accuracy of the network on the 10000 test images: 49 %


In [32]:
path_dr_mnist_network_untrained = './pca_reduced_networks/mnist/pca_mnist_network.pt'
torch.save(pca_mnist_network, path_dr_mnist_network_untrained)


In [33]:
average_accuracy(pca_mnist_network, testloader_mnist)


Accuracy of the network on the 10000 test images: 49 %


## retraining dimensionally reduced network

In [34]:
pca_mnist_network_retrained= copy.deepcopy(pca_mnist_network)

In [35]:
average_accuracy(pca_mnist_network_retrained, testloader_mnist)


Accuracy of the network on the 10000 test images: 49 %


In [36]:
training(pca_mnist_network_retrained, trainloader_mnist,
         validloader_mnist, 10, 'pca_mnist_retrained_accloss')


SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0


c:\Users\sulta\anaconda3\lib\site-packages\torch\autograd\profiler.py:160: UserWarning: CUDA is not available, disabling CUDA profiling
  warn("CUDA is not available, disabling CUDA profiling")


[1,  2500] train loss: nan train acc: 9.902 valid acc: 9.700 valid loss  nan 
epoch: 1


KeyboardInterrupt: 

In [455]:
path_pca_mnsit_network_retrained = './pca_reduced_networks/mnist/pca_mnist.network_retrained.pt'
torch.save(pca_mnist_network_retrained, path_pca_mnsit_network_retrained)


## reconstructing and training model from scratch without dr

In [269]:
small_layer_mnist = (nn.Linear(400,60), nn.Linear(60,42), nn.Linear(42,10))
small_network_mnist = NN(is_norm=False)
small_network_mnist = construct_model(small_network_mnist,small_layer_mnist)

In [456]:
average_accuracy(small_network_mnist, testloader_mnist)

Accuracy of the network on the 10000 test images: 98 %


In [271]:
training(small_network_mnist, trainloader_mnist, 5)


SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0


c:\Users\sulta\anaconda3\lib\site-packages\torch\autograd\profiler.py:160: UserWarning: CUDA is not available, disabling CUDA profiling
  warn("CUDA is not available, disabling CUDA profiling")


[1,  2000] loss: 1.666 acc: 80.963
[1,  4000] loss: 1.486 acc: 95.450
[1,  6000] loss: 1.471 acc: 96.062
[1,  8000] loss: 1.463 acc: 96.425
[1, 10000] loss: 1.456 acc: 97.138
[1, 12000] loss: 1.452 acc: 97.338
[1, 14000] loss: 1.448 acc: 97.550
epoch: 1
[2,  2000] loss: 2.168 acc: 97.850
[2,  4000] loss: 1.444 acc: 97.825
[2,  6000] loss: 1.442 acc: 98.075
[2,  8000] loss: 1.444 acc: 97.787
[2, 10000] loss: 1.441 acc: 98.162
[2, 12000] loss: 1.438 acc: 98.487
[2, 14000] loss: 1.442 acc: 98.037
epoch: 2
[3,  2000] loss: 2.156 acc: 98.450
[3,  4000] loss: 1.436 acc: 98.550
[3,  6000] loss: 1.435 acc: 98.700
[3,  8000] loss: 1.437 acc: 98.475
[3, 10000] loss: 1.436 acc: 98.588
[3, 12000] loss: 1.435 acc: 98.713
[3, 14000] loss: 1.435 acc: 98.562
epoch: 3
[4,  2000] loss: 2.153 acc: 98.558
[4,  4000] loss: 1.433 acc: 98.838
[4,  6000] loss: 1.433 acc: 98.737
[4,  8000] loss: 1.432 acc: 98.838
[4, 10000] loss: 1.432 acc: 98.950
[4, 12000] loss: 1.433 acc: 98.688
[4, 14000] loss: 1.431 acc: 

In [272]:
path_pca_mnsit_network_scratch = './pca_reduced_networks/mnist/small_network_mnist.pt'
torch.save(small_network_mnist, path_pca_mnsit_network_scratch)


In [273]:
average_accuracy(small_network_mnist, testloader_mnist)


Accuracy of the network on the 10000 test images: 98 %


## Pruning mnist net

In [12]:
def prune_function(model, amount_to_prune):
    for name,module in model.named_modules():
        if isinstance(module, nn.Linear):
            prune.random_unstructured(module, 'weight', amount=amount_to_prune)
            print(module._forward_pre_hooks)
        

In [286]:
pruned_mnist_net = copy.deepcopy(original_network)
prune_function(pruned_mnist_net, 0.5)

OrderedDict([(12, <torch.nn.utils.prune.RandomUnstructured object at 0x00000259D73B5640>)])
OrderedDict([(13, <torch.nn.utils.prune.RandomUnstructured object at 0x00000259D73B5580>)])
OrderedDict([(14, <torch.nn.utils.prune.RandomUnstructured object at 0x00000259D73B5A30>)])


In [287]:
average_accuracy(original_network, testloader_mnist)

Accuracy of the network on the 10000 test images: 98 %


In [467]:
average_accuracy(pruned_mnist_net, testloader_mnist)

Accuracy of the network on the 10000 test images: 74 %


In [289]:
path_pruned_net ='./pca_reduced_networks/mnist/pruned/pruned_original_net.pt'
torch.save(pruned_mnist_net, path_pruned_net)

### DR on pruned network

#### pca

In [600]:
weightlist = drprojection(pruned_mnist_net, dec.PCA)


fc1 layer weight
x_new vector
torch.Size([60, 256])
torch.Size([120, 60])
torch.Size([84, 60])
x_new vector
torch.Size([42, 60])
fc2 layer weight
torch.Size([42, 60])
fc3 layer weight
torch.Size([10, 42])


In [601]:
pca_layers = (nn.Linear(256, 60, False),nn.Linear(60, 42, False), nn.Linear(42, 10, False))
pca_pruned_mnist_net = copy.deepcopy(pruned_mnist_net)
pca_pruned_mnist_net = construct_model(pca_pruned_mnist_net,pca_layers)
pca_pruned_mnist_net = are_weights(pca_pruned_mnist_net, tuple(weightlist))
# pca_pruned_mnist_net.fc1.bias.data = biasfc1
# pca_pruned_mnist_net.fc2.bias.data = biasfc2
# pca_pruned_mnist_net.fc3.bias.data = biasfc3


In [602]:
average_accuracy(pca_pruned_mnist_net, testloader_mnist)

Accuracy of the network on the 10000 test images: 25 %


In [603]:
print(torch.max(pca_pruned_mnist_net(sample_im), dim=1))

torch.return_types.max(
values=tensor([0.6993, 0.3183, 0.5232, 0.5873], grad_fn=<MaxBackward0>),
indices=tensor([9, 3, 1, 9]))


In [604]:
path_pruned_pca_net = './pca_reduced_networks/mnist/pruned/pruned_pca_mnist_net.pt'
torch.save(pca_pruned_mnist_net, path_pruned_pca_net)


#### Sparsity pca

 write about how retrieving the bias and dimensioanlly reducing them helps to maintain some infromation however when eliminating the bais off the equation when it comes to network it imporves the overall accuracy of the model.

i used sparsepca to potentially eliminated the sparse elements of the matrix, hence theoretically should imporve the accuracy in constrats to using PCA

in order to stabilise the output of the dimensional reduction and the porjection of the matrices we reconstructed the network structure by normalizing the output, which when compares to original network it can be seen that the cosine similarity is better with normalisation rather than without

In [639]:
sparsepca_layers = (nn.Linear(256, 60, False),nn.Linear(60, 42, False), nn.Linear(42, 10, False))
sparsepca_pruned_mnist_net = copy.deepcopy(pruned_mnist_net)
upsacled_weights = []
for name, module in sparsepca_pruned_mnist_net.named_modules():
    if isinstance(module, nn.Linear):
        upsacled_weights.append(module.weight * 100)

sparsepca_pruned_mnist_net = are_weights(sparsepca_pruned_mnist_net, upsacled_weights)
sparse_weightlist = drprojection(sparsepca_pruned_mnist_net, dec.MiniBatchSparsePCA)

for i in range(len(sparse_weightlist)):
    sparse_weightlist[i] =  torch.divide(sparse_weightlist[i], 100)

sparsepca_pruned_mnist_net = construct_model(
    sparsepca_pruned_mnist_net, sparsepca_layers)
sparsepca_pruned_mnist_net = are_weights(sparsepca_pruned_mnist_net, tuple(sparse_weightlist))


tensor([[ 1.9601, -0.0000, -0.0000,  ...,  3.8460, -0.0000, -0.0000],
        [-0.0000,  1.1808, -0.0000,  ..., -3.3882, -0.0000, -0.0000],
        [ 0.0000,  2.6545, -0.0000,  ..., -3.3634, -0.0000,  4.7944],
        ...,
        [-0.0000, -0.7033,  4.4505,  ...,  0.0000, -4.9641,  0.0000],
        [ 7.4613,  0.4271,  0.0000,  ...,  2.4404, -0.0000, -0.0000],
        [ 0.0000,  0.0000, -5.2224,  ...,  0.8172, -4.9365,  2.2903]])
fc1 layer weight
x_new vector
torch.Size([60, 256])
torch.Size([120, 60])
torch.Size([84, 60])
x_new vector
torch.Size([42, 60])
fc2 layer weight
torch.Size([42, 60])
fc3 layer weight
torch.Size([10, 42])
tensor([[-0.5485, -4.1330, -5.3164,  ...,  7.1366,  5.6381, -0.2025],
        [ 1.6966, -2.1672, -0.2171,  ...,  4.1454, -4.1093, -3.4350],
        [ 3.1742,  1.4251,  8.5305,  ..., -1.8840,  1.1888,  1.9020],
        ...,
        [-0.8448, -5.1814,  4.7656,  ...,  0.7385, -5.5100,  8.2790],
        [-3.4957,  1.7812,  5.4462,  ...,  4.7664,  0.9722,  0.8580]

In [643]:
average_accuracy(sparsepca_pruned_mnist_net, testloader_mnist)

Accuracy of the network on the 10000 test images: 34 %


In [642]:
path_sparse_pca_mnist_network = './pca_reduced_networks/mnist/pruned/sparse_pca_mnist_network.pt'
torch.save(sparsepca_pruned_mnist_net, path_sparse_pca_mnist_network)

# Dimensional reduction on cifar dataset

In [8]:
class NN2(nn.Module):
    def __init__(self, is_norm=False):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3, out_channels=12, kernel_size=(5, 5))
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=12, out_channels=16, kernel_size=(5, 5))
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.soft = nn.Softmax(dim=0)
        self.is_norm = is_norm
        # self.conv1 = nn.Conv2d(3, 6, 5)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = nn.Conv2d(6, 16, 5)
        # self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # self.fc2 = nn.Linear(120, 84)
        # self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        if(self.is_norm == True):
            x = nn.functional.normalize(self.fc3(x))
        else:
            x = self.fc3(x)
        return x


In [647]:
cifar_original_net = NN2(is_norm=True)


In [648]:
training(cifar_original_net, trainloader, 5)

SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0


c:\Users\sulta\anaconda3\lib\site-packages\torch\autograd\profiler.py:160: UserWarning: CUDA is not available, disabling CUDA profiling
  warn("CUDA is not available, disabling CUDA profiling")


[1,  2000] loss: 2.119 acc: 26.150
[1,  4000] loss: 1.991 acc: 38.975
[1,  6000] loss: 1.953 acc: 42.850
[1,  8000] loss: 1.924 acc: 46.562
[1, 10000] loss: 1.908 acc: 47.888
[1, 12000] loss: 1.895 acc: 48.625
epoch: 1
[2,  2000] loss: 2.345 acc: 50.700
[2,  4000] loss: 1.858 acc: 53.350
[2,  6000] loss: 1.864 acc: 52.388
[2,  8000] loss: 1.846 acc: 54.125
[2, 10000] loss: 1.851 acc: 53.763
[2, 12000] loss: 1.829 acc: 55.688
epoch: 2
[3,  2000] loss: 2.268 acc: 57.670
[3,  4000] loss: 1.809 acc: 58.225
[3,  6000] loss: 1.813 acc: 57.650
[3,  8000] loss: 1.806 acc: 58.275
[3, 10000] loss: 1.810 acc: 58.013
[3, 12000] loss: 1.801 acc: 58.938
epoch: 3
[4,  2000] loss: 2.228 acc: 60.650
[4,  4000] loss: 1.782 acc: 61.150
[4,  6000] loss: 1.777 acc: 61.263
[4,  8000] loss: 1.773 acc: 61.712
[4, 10000] loss: 1.779 acc: 61.500
[4, 12000] loss: 1.771 acc: 62.050
epoch: 4
[5,  2000] loss: 2.193 acc: 63.810
[5,  4000] loss: 1.755 acc: 63.800
[5,  6000] loss: 1.753 acc: 64.188
[5,  8000] loss: 1.

In [10]:
path_original_network = './pca_reduced_networks/cifar/cifar_original_net.pt'
cifar_original_net = torch.load(path_original_network) 

In [11]:
average_accuracy(cifar_original_net, trainloader)


Accuracy of the network on the 10000 test images: 69 %


In [651]:
cifar_weights = drprojection(cifar_original_net, dec.PCA)


fc1 layer weight
x_new vector
torch.Size([60, 400])
torch.Size([120, 60])
torch.Size([84, 60])
x_new vector
torch.Size([42, 60])
fc2 layer weight
torch.Size([42, 60])
fc3 layer weight
torch.Size([10, 42])


In [666]:
layers3 = (nn.Linear(400, 60, False), nn.Linear(60, 42, False), nn.Linear(42, 10, False))
dr_cifar_untrained_network = construct_model(cifar_original_net, layers3)
dr_cifar_untrained_network = are_weights(
    dr_cifar_untrained_network, tuple(cifar_weights))


In [667]:
average_accuracy(dr_cifar_untrained_network, testloader)

Accuracy of the network on the 10000 test images: 27 %


In [668]:
path_dr_original_network = './pca_reduced_networks/cifar/cifar_dr_original_net.pt'
torch.save(dr_cifar_untrained_network,path_dr_original_network) 

## dimensioanlly reduced and trained network on cifar10

In [669]:
dr_cifar_trained_network  = copy.deepcopy(dr_cifar_untrained_network)

In [670]:
average_accuracy(dr_cifar_trained_network, testloader)

Accuracy of the network on the 10000 test images: 27 %


In [673]:
training(dr_cifar_trained_network, trainloader, 5)


SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0
[1,  2000] loss: 2.010 acc: 40.188
[1,  4000] loss: 2.006 acc: 39.712
[1,  6000] loss: 2.000 acc: 41.525
[1,  8000] loss: 1.994 acc: 43.175
[1, 10000] loss: 1.987 acc: 43.625
[1, 12000] loss: 1.976 acc: 44.413
epoch: 1
[2,  2000] loss: 2.465 acc: 46.540
[2,  4000] loss: 1.963 acc: 47.125
[2,  6000] loss: 1.964 acc: 47.212
[2,  8000] loss: 1.961 acc: 47.312
[2, 10000] loss: 1.956 acc: 47.550
[2, 12000] loss: 1.949 acc: 48.675
epoch: 2
[3,  2000] loss: 2.426 acc: 49.990
[3,  4000] loss: 1.940 acc: 48.900
[3,  6000] loss: 1.935 acc: 50.900
[3,  8000] loss: 1.936 acc: 50.688
[3, 10000] loss: 1.936 acc: 50.362
[3, 12000] loss: 1.927 acc: 51.538
epoch: 3
[4,  2000] loss: 2.406 acc: 51.290
[4,  4000] loss: 1.909 acc: 53.575
[4,  6000] loss: 1.917 acc: 53.013
[4,  8000] loss: 1.912 acc: 53.575
[4, 10000] loss: 1.906 acc: 54.388
[4, 12

In [674]:
path_net_retrained = 'pca_reduced_networks/cifar/cifar_dr_net_retrained_retrained.pt'
torch.save(dr_cifar_trained_network,path_net_retrained)

## reconstructing model without dimensional redcution 

In [675]:
layers5 = (nn.Linear(400, 60), nn.Linear(60, 42), nn.Linear(42, 10))
reconstruct_network = construct_model(NN2(is_norm=True), layers5)


In [677]:
average_accuracy(reconstruct_network, testloader)


Accuracy of the network on the 10000 test images: 10 %


In [678]:
training(reconstruct_network, trainloader, 5)

SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0


c:\Users\sulta\anaconda3\lib\site-packages\torch\autograd\profiler.py:160: UserWarning: CUDA is not available, disabling CUDA profiling
  warn("CUDA is not available, disabling CUDA profiling")


[1,  2000] loss: 2.155 acc: 23.587
[1,  4000] loss: 2.021 acc: 36.750
[1,  6000] loss: 1.973 acc: 40.562
[1,  8000] loss: 1.945 acc: 44.200
[1, 10000] loss: 1.931 acc: 44.862
[1, 12000] loss: 1.915 acc: 46.337
epoch: 1
[2,  2000] loss: 2.380 acc: 47.630
[2,  4000] loss: 1.892 acc: 48.862
[2,  6000] loss: 1.883 acc: 50.087
[2,  8000] loss: 1.873 acc: 50.763
[2, 10000] loss: 1.857 acc: 52.825
[2, 12000] loss: 1.852 acc: 52.675
epoch: 2
[3,  2000] loss: 2.304 acc: 54.280
[3,  4000] loss: 1.838 acc: 55.050
[3,  6000] loss: 1.826 acc: 56.312
[3,  8000] loss: 1.833 acc: 55.487
[3, 10000] loss: 1.824 acc: 56.612
[3, 12000] loss: 1.823 acc: 56.750
epoch: 3
[4,  2000] loss: 2.249 acc: 59.410
[4,  4000] loss: 1.806 acc: 58.237
[4,  6000] loss: 1.808 acc: 57.950
[4,  8000] loss: 1.794 acc: 59.737
[4, 10000] loss: 1.796 acc: 59.513
[4, 12000] loss: 1.795 acc: 59.300
epoch: 4
[5,  2000] loss: 2.221 acc: 61.840
[5,  4000] loss: 1.782 acc: 61.237
[5,  6000] loss: 1.769 acc: 62.625
[5,  8000] loss: 1.

In [679]:
path_net_retrained_scratch = 'pca_reduced_networks/cifar/net_reconstructed_from_scratch.pt'
torch.save(reconstruct_network,path_net_retrained_scratch)

## Pruning on cifar net

### First we prune the networkj without the dimensional reduction

as cifar dataset is relatively more complex than mnist it is expected to perform more poorly, hence it si expected to have lower accuracy 

In [36]:
pruned_cifar_net = copy.deepcopy(cifar_original_net)
prune_function(pruned_cifar_net, 0.5)

OrderedDict([(9, <torch.nn.utils.prune.RandomUnstructured object at 0x0000013A99639190>)])
OrderedDict([(10, <torch.nn.utils.prune.RandomUnstructured object at 0x0000013A90456F40>)])
OrderedDict([(11, <torch.nn.utils.prune.RandomUnstructured object at 0x0000013A90477820>)])


In [37]:
average_accuracy(pruned_cifar_net, testloader)

Accuracy of the network on the 10000 test images: 39 %


In [38]:
path_prune_cifar_net = './pca_reduced_networks/cifar/pruned/pruned_cifar_net.pt'
torch.save(pruned_cifar_net,path_prune_cifar_net )

### Dimensional reduction on pruned network

#### PCA

In [48]:
pca_pruned_cifar_net = copy.deepcopy(pruned_cifar_net)

In [49]:
pruned_weightlist = drprojection(pca_pruned_cifar_net, dec.PCA)


fc1 layer weight
x_new vector
torch.Size([60, 400])
torch.Size([120, 60])
torch.Size([84, 60])
x_new vector
torch.Size([42, 60])
fc2 layer weight
torch.Size([42, 60])
fc3 layer weight
torch.Size([10, 42])


In [58]:
pca_cifar_layers = (nn.Linear(400, 60,False), nn.Linear(60, 42,False), nn.Linear(42, 10,False))
pca_pruned_cifar_net = construct_model(pca_pruned_cifar_net, pca_cifar_layers)
pca_pruned_cifar_net = are_weights(pca_pruned_cifar_net, tuple(pruned_weightlist))
average_accuracy(pca_pruned_cifar_net, testloader)

Accuracy of the network on the 10000 test images: 26 %


In [59]:
path_pca_pruned_cifar_net = './pca_reduced_networks/cifar/pruned/pruned_pca_cifar_net.pt'
torch.save(pca_pruned_cifar_net, path_pca_pruned_cifar_net)


#### Sparsity pca

In [79]:
sparse_pca_pruned_cifar_net = copy.deepcopy(pruned_cifar_net)

sparsepca_layers_cifar = (nn.Linear(400, 60, False), nn.Linear(
    60, 42, False), nn.Linear(42, 10, False))
upsacled_weights_cifar = []
for name, module in sparse_pca_pruned_cifar_net.named_modules():
    if isinstance(module, nn.Linear):
        upsacled_weights_cifar.append(module.weight * 100)

sparse_pca_pruned_cifar_net = are_weights(
    sparse_pca_pruned_cifar_net, upsacled_weights_cifar)
sparse_weightlist_cifar = drprojection(
    sparse_pca_pruned_cifar_net, dec.MiniBatchSparsePCA)

for i in range(len(sparse_weightlist_cifar)):
    sparse_weightlist_cifar[i] = torch.divide(sparse_weightlist_cifar[i], 100)

sparse_pca_pruned_cifar_net = construct_model(
    sparse_pca_pruned_cifar_net, sparsepca_layers_cifar)
sparse_pca_pruned_cifar_net = are_weights(
    sparse_pca_pruned_cifar_net, tuple(sparse_weightlist_cifar))


fc1 layer weight
x_new vector
torch.Size([60, 400])
torch.Size([120, 60])
torch.Size([84, 60])
x_new vector
torch.Size([42, 60])
fc2 layer weight
torch.Size([42, 60])
fc3 layer weight
torch.Size([10, 42])


In [82]:
average_accuracy(sparse_pca_pruned_cifar_net, testloader)

Accuracy of the network on the 10000 test images: 31 %


In [81]:
path_pruned_sparsepca_cifar_net = './pca_reduced_networks/cifar/pruned/pruned_sparsepca_cifar_net.pt'
torch.save(sparse_pca_pruned_cifar_net, path_pruned_sparsepca_cifar_net)